In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import pandas as pd
import re
from os import listdir
from os.path import isfile, join
from random import shuffle
from collections import Counter
import pandas as pd
import string
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

In [ ]:
import pkg_resources
import nltk

from nltk.corpus import stopwords
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer
from symspellpy import SymSpell # Fuzzy search and word correction

In [ ]:
import importlib
import utils 
from utils import *
import preprocessing
from preprocessing import *

importlib.reload(utils)
importlib.reload(preprocessing)

<module 'utils' from 'c:\\Users\\rayji\\OneDrive\\Documents\\GitHub\\ml-project-2-big-three\\utils.py'>

In [ ]:
# nltk weights
nltk.download("averaged_perceptron_tagger")
nltk.download("stopwords")
nltk.download("wordnet")

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\rayji\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\rayji\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\rayji\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

# Path

In [ ]:
# Global path
GLOBAL_PATH = "C:/Users/rayji/OneDrive/Documents/Projet 2 ML"

# GloVe
GLOVE_PATH = f"{GLOBAL_PATH}/data/glove.twitter.27B.100d.txt"

# Train full
TRAIN_NEG_FULL_PATH = f"{GLOBAL_PATH}/data/train_neg_full.txt"
TRAIN_POS_FULL_PATH = f"{GLOBAL_PATH}/data/train_pos_full.txt"

# Train
TRAIN_NEG_PATH = f"{GLOBAL_PATH}/data/train_neg.txt"
TRAIN_POS_PATH = f"{GLOBAL_PATH}/data/train_pos.txt"

# Test
TEST_PATH = f"{GLOBAL_PATH}/data/test_data.txt"

# Preprocessed data
TRAIN_PREP_PATH = f"{GLOBAL_PATH}/data/preprocessed/train_gru.csv"
TEST_PREP_PATH = f"{GLOBAL_PATH}/data/preprocessed/test_gru.csv"

# Weight
WEIGHT_PATH = f"{GLOBAL_PATH}/weight"

In [ ]:
!pip install symspellpy

^C


# Load preprocessed dataset

In [1]:
from utils import *
import os

In [2]:
Preprocessed_full_dataset = load_preprocessed_data(os.getcwd() + "/twitter-datasets/train_preprocessed_full.csv")

In [3]:
Preprocessed_full_dataset

,text,label
0,vinco tres or pack 6 ( difficulty 10 of 10 obj...,-1
1,glad i dot have taks tomorrow ! ! thankful ...,-1
2,1-3 vs celtics in the regular season were fu...,-1
3,user i could actually kill that girl i m so s...,-1
4,user user user i find that very hard to b...,-1
...,...,...
2270477,a warning sign ( rt user the negativity y...,1
2270478,user ff too thank youuu ) ),1
2270479,i just love shumpa ! that s my girl,1
2270480,the best way to start a day ! no matter what h...,1


In [4]:
import os

current_folder = os.getcwd()
print(current_folder)


c:\Users\rayji\OneDrive\Documents\GitHub\ml-project-2-big-three


# Abstract method

In [2]:
from sklearn.model_selection import train_test_split
from abc import ABC, abstractmethod
import pandas as pd

In [3]:
class AbstractModel(ABC):
    def __init__(self, weights_path: str):
        self.__weights_path = weights_path


    @abstractmethod
    def get_preprocessing_methods(self, is_test: bool = False):
        pass


    @abstractmethod
    def fit_predict(self, X, y, ids_test, X_test, prediction_path):
        pass


    @abstractmethod
    def predict(self, ids, X, path):
        pass


    @staticmethod
    def _create_submission(ids: list[int], predictions: list[int], path: str):
        # Generating the submission file
        submission = pd.DataFrame(columns=["Id", "Prediction"],
                                data={"Id": ids, "Prediction": predictions})

        # For many models the labels are 0 or 1. Replacing 0s with -1s.
        submission["Prediction"].replace(0, -1, inplace=True)

        # Saving the file
        submission.to_csv(path, index=False)


    @staticmethod
    def _split_data(X: pd.DataFrame, y: pd.DataFrame, test_size: float = 0.2, random_state: int = 42, **kwargs) -> tuple:
        print("Splitting data in train and test set...")
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=random_state, **kwargs)

        return X_train, X_test, y_train, y_test

# Preprocessing


In [1]:
!pip install symspellpy

^C


In [4]:
import pkg_resources
import nltk
import re
import pandas as pd
import numpy as np

from nltk.corpus import stopwords
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer
from symspellpy import SymSpell # Fuzzy search and word correction

In [5]:
# nltk weights
nltk.download("averaged_perceptron_tagger")
nltk.download("stopwords")
nltk.download("wordnet")

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\rayji\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\rayji\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\rayji\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [6]:
import importlib
import preprocessing
from preprocessing import *
# Reload the library
importlib.reload(preprocessing)

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\rayji\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\rayji\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\rayji\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\rayji\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\rayji\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\rayji\AppData\Roaming\nltk_data.

<module 'preprocessing' from 'c:\\Users\\rayji\\OneDrive\\Documents\\GitHub\\ml-project-2-big-three\\preprocessing.py'>

In [ ]:
methods = [
    'drop_duplicates',
    'correct_spacing_indexing',
    'remove_parentheses',
    'correct_spacing_indexing',
    'word_segmentation',
    'correct_spacing_indexing',
    'correct_spacing_indexing',
    'remove_selected_characters',
    'hashtags_to_tags',
    ]

In [ ]:
def run_preprocessing(method_list, full_data=True):
    """
    Runs the preprocessing methods according to the chosen classifier
      on the train and test data

    :param csr: chosen classifier (child of AbstractModel)
    :type csr: AbstractModel
    :param train_preprocessed_path: path to load train data
    :type train_preprocessed_path: str
    :param test_preprocessed_path: path to load test data
    :type test_preprocessed_path: str
    :param full_data: if False, the small dataset (200K rows) is used
    :type full_data: bool, optional
    """

    # Read data
    if full_data:
        dataset_files = [TRAIN_NEG_FULL_PATH, TRAIN_POS_FULL_PATH]
    else:
        dataset_files = [TRAIN_NEG_PATH, TRAIN_POS_PATH]

    train_preprocessing = Preprocessing(dataset_files, is_test=False)
    test_preprocessing = Preprocessing([TEST_PATH], is_test=True)

    # Preprocess it

    methods = method_list
    
    for method in methods:
        getattr(test_preprocessing, method)()

    methods.append('drop_duplicates')


    for method in methods:
        getattr(train_preprocessing, method)()

    train_df = train_preprocessing.__get__()
    #train_df.to_csv(train_preprocessed_path, index=False)

    test_df = test_preprocessing.__get__()
    #test_df.to_csv(test_preprocessed_path, index=False)

    return train_df, test_df

In [16]:
train_preprocessed_dataset, test_preprocessed_dataset = run_preprocessing(full_data=True)
train_preprocessed_dataset.to_csv('C:/Users/rayji/OneDrive/Documents/GitHub/ml-project-2-big-three/data/preprocessed/train_preprocessed_full.csv', index=False)
test_preprocessed_dataset.to_csv('C:/Users/rayji/OneDrive/Documents/GitHub/ml-project-2-big-three/data/preprocessed/test_preprocessed.csv', index=False)

Executing: `drop_duplicates`
Executing: `correct_spacing_indexing`
Correcting spacing...
Executing: `remove_parentheses`
Executing: `correct_spacing_indexing`
Correcting spacing...
Executing: `word_segmentation`


100%|██████████| 2270482/2270482 [05:15<00:00, 7194.59it/s] 


Executing: `correct_spacing_indexing`
Correcting spacing...
Executing: `correct_spacing_indexing`
Correcting spacing...
Executing: `remove_selected_characters`
Removing selected characters...
Executing: `hashtags_to_tags`
Converting hashtags to tags...
Executing: `drop_duplicates`
Executing: `correct_spacing_indexing`
Correcting spacing...
Executing: `remove_parentheses`
Executing: `correct_spacing_indexing`
Correcting spacing...
Executing: `word_segmentation`


100%|██████████| 10000/10000 [00:05<00:00, 1800.45it/s]


Executing: `correct_spacing_indexing`
Correcting spacing...
Executing: `correct_spacing_indexing`
Correcting spacing...
Executing: `remove_selected_characters`
Removing selected characters...
Executing: `hashtags_to_tags`
Converting hashtags to tags...
Executing: `drop_duplicates`
Executing: `__get__`
Executing: `__get__`


In [17]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(train_preprocessed_dataset['text'], train_preprocessed_dataset['label'], test_size=0.2, random_state=42)

# Create a TF-IDF vectorizer
vectorizer = TfidfVectorizer()

# Transform the text data into TF-IDF features
X_train_tfidf = vectorizer.fit_transform(X_train)
X_test_tfidf = vectorizer.transform(X_test)

# Train a logistic regression model
model = LogisticRegression(max_iter=100000)
model.fit(X_train_tfidf, y_train)

# Predict labels for the test set
y_pred = model.predict(X_test_tfidf)

# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

Accuracy: 0.8106527900426561


In [12]:
from nltk.tokenize import word_tokenize
from nltk.corpus import wordnet
from nltk.corpus import words
from collections import Counter


# Download the English language dictionary
nltk.download('words')
nltk.download('punkt')


texts = test_preprocessed_dataset['text']

# Function to clean and tokenize text
def clean_and_tokenize(text):
    # Remove non-alphabetic characters (keep only letters, parentheses, numbers, '.', '-', and '!')
    clean_text = re.sub(r'[^a-zA-Z0-9.!\-()]', ' ', text)
    # Tokenize
    return word_tokenize(clean_text.lower())

# Tokenize the text
tokens = [clean_and_tokenize(text) for text in texts]
flat_tokens = [item for sublist in tokens for item in sublist]


# Load a set of standard English words
english_words = set(words.words())

# Filter out standard English words
non_standard_tokens = [word for word in flat_tokens if word not in english_words]

# Count the frequencies
word_freq = Counter(non_standard_tokens)

# You can define a threshold for what you consider 'high frequency'
high_freq_threshold = 10  # Example threshold
high_freq_words = {word: count for word, count in word_freq.items() if count > high_freq_threshold}

ordered_words = {word: count for word, count in sorted(high_freq_words.items(), key=lambda item: item[1], reverse=True)}
print(ordered_words)

[nltk_data] Downloading package words to
[nltk_data]     C:\Users\rayji\AppData\Roaming\nltk_data...
[nltk_data]   Package words is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\rayji\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


{'!': 3856, '.': 3161, 'url': 2073, '...': 2062, '(': 1985, '-': 827, 'rt': 536, '..': 458, '3': 391, 'lol': 388, ')': 364, '2': 286, 'im': 259, 'haha': 252, '1': 173, '4': 159, 'xxx': 133, 'xx': 131, 'omg': 113, 'fuck': 103, '5': 96, 'guys': 94, 'friends': 87, '6': 83, 'ok': 79, 'okay': 79, 'shit': 75, '10': 74, '8': 73, 'girls': 70, 'dvd': 67, 'friday': 65, 'cd': 63, 'aww': 62, '20': 60, 'things': 59, 'lmao': 58, 'hahaha': 58, 'fans': 58, 'makes': 56, 'looks': 49, 'anymore': 47, '7': 47, 'tweets': 47, 'followers': 46, 'missed': 46, 'hours': 46, 'laptop': 46, 'hardcover': 45, 'wanted': 44, 'boys': 43, 'wants': 43, '12': 41, 'sooo': 41, 'awww': 41, 'sucks': 39, 'proud': 39, 'mom': 38, '18': 38, 'called': 38, 'followed': 36, 'hp': 36, 'women': 34, 'fucking': 34, 'soo': 34, 'tv': 33, '100': 33, 'saturday': 32, 'manuf': 32, 'years': 32, 'justin': 32, 'manufactured': 31, '30': 31, 'gets': 31, 'boyfriend': 31, 'lil': 29, 'hurts': 29, 'books': 29, 'cuz': 29, 'mini': 29, '1gb': 28, 'dm': 28, 

In [124]:
import pandas as pd

# Convert the dictionary to a DataFrame
df_ordered_words = pd.DataFrame.from_dict(ordered_words, orient='index', columns=['Count'])

# Export the DataFrame to a CSV file
df_ordered_words.to_csv('data/preprocessed/ordered_words.csv', index_label='Word')
